Leung Wai Liu <br>
JPMorgan AI Research<br>
August 8, 2022 <br>
Twitter Relabelling VALIDATION - VERSION 2

In [15]:
import pandas as pd
import numpy as np
import wordninja
import spacy
from collections import Counter
from labels_to_tweebank import labels_to_tweebank

In [16]:
nlp = spacy.load('en_core_web_trf')

In [17]:
nlp.get_pipe("ner").labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')

In [18]:
combined_df = pd.read_csv('logs/combined_val_labels_FINAL.tsv', sep='\t')
combined_df

,Word,Start Word Index,End Word Index,POS Label,NER Label,Tweet,Tweet POS Labels,Tweet NER Labels
0,tc,0,0,['PROPN'],['O'],"tc : facebook ’s timeline is changing again , ...","['PROPN', 'PUNCT', 'PROPN', 'PART', 'NOUN', 'A...","['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', '..."
1,@user1373,3,3,['PROPN'],['O'],rt @user1836 : @user1373 @user182 at a minimum...,"['X', 'X', 'PUNCT', 'PROPN', 'PROPN', 'ADP', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,@user182,4,4,['PROPN'],['O'],rt @user1836 : @user1373 @user182 at a minimum...,"['X', 'X', 'PUNCT', 'PROPN', 'PROPN', 'ADP', '...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,@user2559,5,5,['PROPN'],['O'],1 week to go ! @user2559 full weekend tickets ...,"['NUM', 'NOUN', 'PART', 'VERB', 'PUNCT', 'PROP...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,@user505,2,2,['PROPN'],['O'],"!!!!! "" @user505 : #yoursexisweak if i fall as...","['PUNCT', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'S...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
...,...,...,...,...,...,...,...,...
430,light,11,11,['PROPN'],['O'],2500 lumen led flashlight cree xml t6 torch re...,"['NUM', 'PROPN', 'PROPN', 'NOUN', 'PROPN', 'PR...","['O', 'O', 'O', 'O', 'B-ORG', 'O', 'O', 'O', '..."
431,@user1695,0,0,['PROPN'],['O'],@user1695 i miss you sm wtf,"['PROPN', 'PRON', 'VERB', 'PRON', 'ADV', 'INTJ']","['O', 'O', 'O', 'O', 'O', 'O']"
432,tueh,10,10,['PROPN'],['O'],rt @user61 : in the capital ! in 2016 ! tueh ....,"['X', 'X', 'PUNCT', 'ADP', 'DET', 'NOUN', 'PUN...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
433,touch ups,4,5,"['PROPN', 'PROPN']","['O', 'O']",i 've just unlocked touch ups achievement in p...,"['PRON', 'AUX', 'ADV', 'VERB', 'PROPN', 'PROPN...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MI..."


In [19]:
new_label = combined_df['NER Label'].tolist()
new_label_before_convert = list(new_label)
new_pos = combined_df['POS Label'].tolist()
new_pos_before_convert = list(new_pos)
new_stand_out_tweet = combined_df['Tweet'].tolist()

for index, row in combined_df.iterrows():
    
    # retrieving the the information of a particular row
    particular_word = row['Word']
    particular_start_index = row['Start Word Index']
    particular_end_index = row['End Word Index'] + 1
    num_words_particular = particular_end_index - particular_start_index
    original_sentence = row['Tweet']

    # splitting the original sentence
    original_sentence_split = original_sentence.split(' ')

    # putting the brackets in the original sentence to signify word phrase:
    word_phrase_stand_out = list(original_sentence_split)
    word_phrase_stand_out[particular_start_index] = "<[" + word_phrase_stand_out[particular_start_index]
    word_phrase_stand_out[particular_end_index - 1] = word_phrase_stand_out[particular_end_index - 1] + "]>"
    word_phrase_stand_out_string = ' '.join(word_phrase_stand_out)
    new_stand_out_tweet[index] = word_phrase_stand_out_string
    
    
    print('-' * 50)
    print('Original Particular word')
    print(particular_word)
    print("Original Sentence Split")     
    print(original_sentence_split)
    print("Original Start Index")
    print(particular_start_index)

    # setting up the initial parameters  
    char_index = 0
    iterate_word_index = 0
    is_hashtag = False
    is_user = False
    encode = ''
    encode_len = 0

    print("FINDING ENTITY OF WORD")

    pre_processed_sentence_split = list(original_sentence_split)
    pre_processed_word_start_index = particular_start_index
    pre_processed_word_end_index = particular_end_index
    pre_processed_char_start_index = 0
    pre_processed_char_start_index_mushed = 0
    pre_processed_sentence_split_string = ""

    # NO SPLITTING, FEEDING @USER TAG IN IT

    word_index = 0 
    while word_index in range(len(pre_processed_sentence_split)):
        word_in_sentence = pre_processed_sentence_split[word_index]
        check_for_hashtag = word_in_sentence[0:1]

        # if hashtag, remove the hashtag
        if check_for_hashtag == '#':
            word_in_sentence = word_in_sentence[1:]
            pre_processed_sentence_split[word_index] = word_in_sentence
        
        to_add_word_string_length = len(word_in_sentence)

        word_index += 1
    
    

    pre_processed_sentence_split_string = ' '.join(pre_processed_sentence_split)

    # Getting the specific character of the post pre-processed sentence
    part_of_sentence_before_particular_word = pre_processed_sentence_split[:pre_processed_word_start_index]
    part_of_sentence_before_particular_word_string = ' '.join(part_of_sentence_before_particular_word)

    pre_processed_char_start_index = (len(part_of_sentence_before_particular_word_string)) + 1

    # Without getting the string without spaces
    part_of_sentence_before_particular_word_string_mushed = ''.join(part_of_sentence_before_particular_word)
    pre_processed_char_start_index_mushed = (len(part_of_sentence_before_particular_word_string_mushed))

    # processing particular word phrase after pre-processing
    particular_word_after_processing = pre_processed_sentence_split[pre_processed_word_start_index: pre_processed_word_end_index]
    particular_word_string_after_processing = ''.join(particular_word_after_processing)
    len_particular_word_string_after_processing_mushed = len(particular_word_string_after_processing)

    # encoding into spacy document to get the NER entity of each word
    sentence_doc = nlp(pre_processed_sentence_split_string) # splitted word phrase
    word_entities = [(e.text, e.ent_iob_, e.ent_type_, e.pos_) for e in sentence_doc]

    # check if the length of the entities that fed in matches with the len of entities that came out
    
    print("FOR NON COMBINED WORDS")

    print("After processing")
    print(pre_processed_sentence_split)
    print("After processing joint sentence")
    print(pre_processed_sentence_split_string)
    print("Pre processed word start index")
    print(pre_processed_word_start_index)
    print("Pre processed char start index")
    print(pre_processed_char_start_index)
    print("Pre processed split string")
    print(pre_processed_sentence_split_string)
    print("Part of sentence before particular word")
    print(part_of_sentence_before_particular_word)
    print("Part of sentence before particular word string")
    print(part_of_sentence_before_particular_word_string)
    print("Part of sentence before particular word string mushed")
    print(part_of_sentence_before_particular_word_string_mushed)
    print("FINAL: Pre Processed char start index mushed")
    print(pre_processed_char_start_index_mushed)    
    print("LEN OF WORD PHRASE AFTER MUSHED")
    print(len_particular_word_string_after_processing_mushed)
    
    print('Word Entities result')
    print(word_entities)

    # retrieving the entities to verify:

    # NON COMBINED SENTENCE 
    entities_index_non_combined = 0
    entities_char_non_combined = 0

    word_entities_to_verify_non_combined = []
    
    for entity in word_entities:
        if entities_char_non_combined < pre_processed_char_start_index_mushed:
            particular_entity_word = entity[0]
            particular_entity_word_length = len(particular_entity_word)
            entities_index_non_combined += 1
            entities_char_non_combined += particular_entity_word_length
        
    
    while entities_char_non_combined < (pre_processed_char_start_index_mushed + len_particular_word_string_after_processing_mushed):
        entity_to_append_non_combined = word_entities[entities_index_non_combined]
        word_entities_to_verify_non_combined.append(entity_to_append_non_combined)
        
        entities_char_non_combined += len(entity_to_append_non_combined[0])
        entities_index_non_combined += 1
    
    print("PRINTING NON COMBINED")
    print(word_entities_to_verify_non_combined)

    # retrieving all the labels and pos tags from each phrase (if any)
    label = []
    pos = []
    
    # otherwise append all of the NER tags from the split word entity (if there are)
    for numents in range(len(word_entities_to_verify_non_combined)):
        if word_entities_to_verify_non_combined[numents][2] != '':
            label.append(word_entities_to_verify_non_combined[numents][2])
        pos.append(word_entities_to_verify_non_combined[numents][3])
    
    print(label)
    print(pos)

    if label != []:
        new_label_before_convert[index] = list(label)
    else:
        new_label_before_convert[index] = []

    new_pos_before_convert[index] = list(pos)
    
    # Filtering - if the label list is not empty: 
    if label != []:
        label = [labels_to_tweebank[label_item] for label_item in label]
        
        if len(label) == num_words_particular:
            for num_word in range(len(label)):
                if num_word == 0:
                    label[num_word] = 'B-' + label[num_word]
                else:
                    label[num_word] = 'I-' + label[num_word]
        elif len(label) > num_words_particular:
            for num_word in range(num_words_particular):
                if num_word == 0:
                    label[num_word] = 'B-' + label[num_word]
                else:
                    label[num_word] = 'I-' + label[num_word]
        elif len(label) < num_words_particular:
            label_counter = Counter(label)
            most_common_label = label_counter.most_common(1)[0][0]
            label = []
            for num_word in range(num_words_particular):
                if num_word == 0:
                    label.append('B-' + most_common_label)
                else:
                    label.append('I-' + most_common_label)
        
        new_label[index] = label
        new_pos[index] = pos

    else:
        if '@user' in particular_word:
            new_label[index] = ['B-PER']
            new_pos[index] = pos
        elif 'PROPN' in pos:
            label = []
            for num_word in range(num_words_particular):
                if num_word == 0:
                    label.append('B-' + 'MISC')
                else:
                    label.append('I-' + 'MISC')
            new_label[index] = label
            new_pos[index] = pos
        else:
            new_pos[index] = pos
    
 


--------------------------------------------------
Original Particular word
tc
Original Sentence Split
['tc', ':', 'facebook', '’s', 'timeline', 'is', 'changing', 'again', ',', 'now', 'prioritises', 'sites', 'you', '’ll', 'actually', 'read', 'url445']
Original Start Index
0
FINDING ENTITY OF WORD
FOR NON COMBINED WORDS
After processing
['tc', ':', 'facebook', '’s', 'timeline', 'is', 'changing', 'again', ',', 'now', 'prioritises', 'sites', 'you', '’ll', 'actually', 'read', 'url445']
After processing joint sentence
tc : facebook ’s timeline is changing again , now prioritises sites you ’ll actually read url445
Pre processed word start index
0
Pre processed char start index
1
Pre processed split string
tc : facebook ’s timeline is changing again , now prioritises sites you ’ll actually read url445
Part of sentence before particular word
[]
Part of sentence before particular word string

Part of sentence before particular word string mushed

FINAL: Pre Processed char start index mushed
0
L

In [20]:
new_df = combined_df
new_df['Spacy NER Label'] = new_label_before_convert
new_df['Spacy POS Label'] = new_pos_before_convert

new_df['Final NER Label'] = new_label
new_df['Final POS Label'] = new_pos

new_df['Candidate Label'] = new_stand_out_tweet

new_df = new_df.reindex(columns = ['Word', 'Start Word Index', 'End Word Index', 'POS Label', 'Spacy POS Label', 'Final POS Label', 'NER Label', 'Spacy NER Label', 'Final NER Label', 'Tweet', 'Candidate Label'])
new_df

,Word,Start Word Index,End Word Index,POS Label,Spacy POS Label,Final POS Label,NER Label,Spacy NER Label,Final NER Label,Tweet,Candidate Label
0,tc,0,0,['PROPN'],[X],[X],['O'],[],['O'],"tc : facebook ’s timeline is changing again , ...",<[tc]> : facebook ’s timeline is changing agai...
1,@user1373,3,3,['PROPN'],[X],[X],['O'],[],[B-PER],rt @user1836 : @user1373 @user182 at a minimum...,rt @user1836 : <[@user1373]> @user182 at a min...
2,@user182,4,4,['PROPN'],[X],[X],['O'],[],[B-PER],rt @user1836 : @user1373 @user182 at a minimum...,rt @user1836 : @user1373 <[@user182]> at a min...
3,@user2559,5,5,['PROPN'],[X],[X],['O'],[],[B-PER],1 week to go ! @user2559 full weekend tickets ...,1 week to go ! <[@user2559]> full weekend tick...
4,@user505,2,2,['PROPN'],[PROPN],[PROPN],['O'],[],[B-PER],"!!!!! "" @user505 : #yoursexisweak if i fall as...","!!!!! "" <[@user505]> : #yoursexisweak if i fal..."
...,...,...,...,...,...,...,...,...,...,...,...
430,light,11,11,['PROPN'],[NOUN],[NOUN],['O'],[],['O'],2500 lumen led flashlight cree xml t6 torch re...,2500 lumen led flashlight cree xml t6 torch re...
431,@user1695,0,0,['PROPN'],[PROPN],[PROPN],['O'],[],[B-PER],@user1695 i miss you sm wtf,<[@user1695]> i miss you sm wtf
432,tueh,10,10,['PROPN'],[X],[X],['O'],[],['O'],rt @user61 : in the capital ! in 2016 ! tueh ....,rt @user61 : in the capital ! in 2016 ! <[tueh...
433,touch ups,4,5,"['PROPN', 'PROPN']","[NOUN, NOUN]","[NOUN, NOUN]","['O', 'O']",[],"['O', 'O']",i 've just unlocked touch ups achievement in p...,i 've just unlocked <[touch ups]> achievement ...


In [21]:
new_df.to_csv('logs/VAL_relabelled_data_spacy_pass_entire_sentence_no_break_apart.tsv', sep='\t', index=False)